# Обзор Asyncio

Asyncio предоставляет еще один инструмент для параллельного программирования на Python, который легче, чем потоки или многопроцессорность. В очень простом смысле это достигается за счет того, что цикл событий выполняет набор задач, с ключевым отличием в том, что каждая задача выбирает, когда передать управление циклу событий.

API asyncio в Python сложен, потому что он нацелен на решение разных проблем для разных групп людей. К сожалению, доступно очень мало рекомендаций, которые помогут вам выяснить, какие части asyncio важны для вашей группы.

Есть две основные группы юзеров Asyncio:
1. Конечные разработчики - Они хотят создавать приложения, использующие asyncio.
2. Разработчики фреймворков - Они хотят создавать фреймворки и библиотеки, которые разработчики конечных пользователей могут использовать в своих приложениях.

Большая часть путаницы вокруг asyncio в современном сообществе происходит из-за непонимания этой разницы. Например, официальная документация Python для asyncio больше подходит для разработчиков фреймворков, чем для конечных пользователей. Это означает, что разработчики конечных пользователей, читающие эти документы, быстро приходят в шок от очевидной сложности. Вы в некоторой степени вынуждены все это осмыслить, прежде чем сможете что-либо с этим сделать.

Я надеюсь, что эта книга поможет вам отличить функции Asyncio, которые важны для разработчиков конечных пользователей, и те, которые важны для разработчиков фреймворков.

# Quickstart

Небольшую часть всего asyncio API, которую можно резюмировать следующим образом:
1. Запустить asyncio event loop (цикл событий).
2. Вызвать async/await функцию.
3. Создать задачу (task), выполняющуюся в цикле.
4. Подождать, пока множество задач выполнятся.
5. Завершить цикл после завершения всех конкурентных задач.

В этом разделе мы рассмотрим эти основные функции и увидим, как выйти из цикла с помощью программирования на основе событий на Python.

### Код запускать в pycharm

In [1]:
import asyncio, time

# патчим asyncio, чтобы небыло ошибок в jupyter. В pycharm этого делать
# не надо
import nest_asyncio
nest_asyncio.apply()

In [7]:
async def main():
    print(f'{time.ctime()} Hello!')
    await asyncio.sleep(1)
    print(f'{time.ctime()} Goodbye!')

In [8]:
asyncio.run(main())

Sat Jan  2 17:30:21 2021 Hello!
Sat Jan  2 17:30:22 2021 Goodbye!


In [4]:
#The “Hello-ish World” of Asyncio
loop = asyncio.get_event_loop() #1
task = loop.create_task(main()) #2
loop.run_until_complete(task) #3
pending = asyncio.all_tasks(loop=loop)
for task in pending:
    task.cancel()
group = asyncio.gather(*pending, return_exceptions=True) #4
loop.run_until_complete(group) #5
loop.close() #6

Sat Jan  2 17:30:03 2021 Hello!
Sat Jan  2 17:30:04 2021 Goodbye!


RuntimeError: Cannot close a running event loop

1. Вам нужен экземпляр цикла, прежде чем вы сможете запускать какие-либо сопрограммы, и вот как вы его получаете. Фактически, где бы вы его ни вызывали, get_event_loop () будет каждый раз выдавать один и тот же экземпляр цикла, если вы используете только один поток. Если вы
внутри функции async def вместо этого следует вызвать asyncio.get_running_loop (), который всегда дает то, что вы ожидаете. Более подробно об этом будет рассказано далее в книге.
2. В данном случае конкретным вызовом является loop.create_task (main ()). Ваша функция сопрограммы не будет выполняться, пока вы этого не сделаете. Мы говорим, что create_task () планирует запуск вашей сопрограммы в цикле. Возвращенный объект задачи может использоваться для отслеживания состояния задачи (например, выполняется ли она еще или завершена), а также может использоваться для получения значения результата из завершенной сопрограммы. Вы можете отменить задачу с помощью task.cancel ().
3. Этот вызов заблокирует текущий поток, который обычно является основным потоком. Обратите внимание, что run_until_complete () будет поддерживать цикл только до тех пор, пока данное сопутствующее событие не завершится, но все другие задачи, запланированные в цикле, также будут выполняться во время выполнения цикла. Внутри asyncio.run () вызывает run_until_complete () для вас и, следовательно, таким же образом блокирует основной поток.
4. Когда «основная» часть программы разблокируется из-за приема сигнала процесса или из-за остановки цикла некоторым кодом, вызывающим loop.stop (), запускается код после run_until_complete (). Стандартная идиома, показанная здесь, состоит в том, чтобы собрать все еще ожидающие задачи, отменить их, а затем снова использовать loop.run_until_complete (), пока эти задачи не будут выполнены. gather () - это метод сбора. Обратите внимание, что asyncio.run () выполняет все отмены, сбор и ожидание завершения отложенных задач.
5. loop.close () обычно является последним действием: он должен вызываться в остановленном цикле, он очищает все очереди и завершает работу исполнителя. Остановленный цикл можно перезапустить, но замкнутый цикл исчезнет навсегда. Внутренне asyncio.run () закроет цикл перед возвратом. Это нормально, потому что run () создает новый цикл событий каждый раз, когда вы его вызываете.

Пример 3

In [5]:
# базовый интерфейс исполнителя
async def main():
    print(f'{time.ctime()} Hello!')
    await asyncio.sleep(1.0)
    print(f'{time.ctime()} Goodbye!')


def blocking(): #1
    time.sleep(0.5) #2
    print(f'{time.ctime()} Hello from a thread')

In [6]:
loop = asyncio.get_event_loop()
task = loop.create_task(main())

loop.run_in_executor(None, blocking) #3
loop.run_until_complete(task)

pending = asyncio.all_tasks(loop=loop) #4
for task in pending:
    task.cancel()
group = asyncio.gather(*pending, return_exceptions=True)
loop.run_until_complete(group)
loop.close()

Sat Jan  2 17:30:10 2021 Hello!
Sat Jan  2 17:30:11 2021 Hello from a thread
Sat Jan  2 17:30:11 2021 Goodbye!


RuntimeError: Cannot close a running event loop

1. blocking () вызывает традиционный time.sleep () внутри, что заблокировало бы основной поток и предотвратило бы запуск вашего цикла событий. Это означает, что вы не должны делать эту функцию сопрограммой - действительно, вы даже не можете вызвать эту функцию из любого места в основном потоке, где выполняется цикл asyncio. Мы решаем эту проблему, запустив эту функцию в исполнителе.
2. Не имеет отношения к этому разделу, но о чем следует помнить позже в книге: обратите внимание, что время блокирующего сна (0,5 секунды) короче, чем время неблокирующего сна (1 секунда) в сопрограмме main (). Это делает пример кода аккуратным и аккуратным. В разделе «Ожидание исполнителя во время завершения работы» на странице 68 мы рассмотрим, что произойдет, если функции исполнителя переживут свои асинхронные аналоги во время завершения работы.
3. Это последний из нашего списка важных функций asyncio, которые необходимо знать. Иногда вам нужно запускать что-то в отдельном потоке или даже в отдельном процессе: этот метод используется именно для этого. Здесь мы передаем нашу функцию блокировки для запуска в исполнителе по умолчанию. Обратите внимание, что run_in_executor () не блокирует основной поток: он только планирует выполнение задачи исполнителя (он возвращает Future, что означает, что вы можете ожидать его, если метод вызывается в другой функции сопрограммы). Задача исполнителя начнет выполняться только после вызова run_until_complete (), что позволяет циклу обработки событий начать обработку событий.
4. В дополнение к примечанию во фрагменте 2: набор задач в ожидании не включает запись для вызова blocking (), сделанного в run_in_executor (). Это будет верно для любого вызова, который возвращает Future, а не Task. Документация достаточно хороша для определения возвращаемых типов, поэтому вы увидите там возвращаемый тип; просто помните, что all_tasks () действительно возвращает только задачи, а не фичи.